In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.models import Sequential

## Simulate data

In [ ]:
t = np.arange(0,1000,step=0.01)
size = t.shape[0]

amp = 1
f = 1
phi = 0
offset = 0
z_clean = amp * np.sin(2*np.pi*f*t+phi) + offset

noise = 0.25*np.random.randn(size)
z = z_clean + noise

In [ ]:
split = int(0.9*size)
t_before = t[:split]
t_after = t[split:]
z_before = z[:split]
z_after = z[split:]

In [ ]:
plt.plot(t_before, z_before)
plt.plot(t_after, z_after)
plt.plot(t, z_clean)
plt.xlim(t_before.max() - 10, t_after.min() + 10)

## Split data

In [ ]:
feature_length = 20

In [ ]:
data = np.array([z[i:i+feature_length+1] for i in tqdm(range(z.shape[0] - feature_length))])

In [ ]:
x_before = data[:split, :feature_length]
x_after = data[split:, :feature_length]

In [ ]:
y_before = data[:split, feature_length:].flatten()
y_after = data[split:, feature_length:].flatten()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_before, y_before, train_size=0.9)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

## Train model

In [ ]:
def cnn_1d():

    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(feature_length,1)))
    model.add(MaxPool1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='Adam', loss='mse', metrics=['mae'])
              
    return model

In [ ]:
model = cnn_1d()

In [ ]:
model.summary()

In [ ]:
baseline = np.mean(np.square(x_train.mean(1) - y_train))

In [ ]:
baseline_z = np.mean(np.square(z - z_clean))

In [ ]:
baseline, baseline_z

In [ ]:
model_history = model.fit(x_train, y_train, epochs=50, verbose=1, validation_split=0.8, batch_size=256)

## Evaluate model

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].set_title('mse')
axs[0].plot(model_history.history['loss'], label='train')
axs[0].plot(model_history.history['val_loss'], label='val')
axs[0].hlines(baseline, 0, 50, color='C2', label='baseline (data)')
axs[0].hlines(baseline_z, 0, 50, color='C3', label='baseline (best_fit)')
#axs[0].set_yscale('log')
axs[0].legend()
axs[1].set_title('mae')
axs[1].plot(model_history.history['mae'], label='train')
axs[1].plot(model_history.history['val_mae'], label='val')
#axs[1].set_yscale('log')
axs[1].legend()
plt.tight_layout()

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.evaluate(x=x_after, y=y_after)

In [ ]:
y_pred = model.predict(x_test).flatten()

In [ ]:
y_pred_after = model.predict(x_after).flatten()

In [ ]:
ind = 1000
plt.plot(x_test[ind], label='feature')
plt.scatter(feature_length+1, y_test[ind], color='C1', label='label')
plt.scatter(feature_length+1, y_pred[ind], color='C2', label='pred')
plt.legend()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=[10,5])
axs[0].hist(y_pred-y_test, alpha=0.5)
axs[0].hist(y_pred_after-y_after, alpha=0.5)
axs[1].scatter(np.arange(y_pred.shape[0]), y_pred-y_test, s=1, alpha=0.1)
axs[1].scatter(np.arange(y_after.shape[0]), y_pred_after-y_after, s=1, alpha=0.1)

In [ ]:
plt.scatter(y_pred, y_test, s=1, alpha=0.1)
plt.scatter(y_pred_after, y_after, s=1, alpha=0.1)

In [ ]:
np.corrcoef(y_pred, y_test)[0,1], np.corrcoef(y_pred_after, y_after)[0,1]

## Recursive fit

In [ ]:
ind = 0
fit = []
rec_fit = list(x_after[ind])
for i in range (feature_length):
    rec_pred = model.predict(np.array(rec_fit[-feature_length:]).reshape(1,feature_length), verbose=0)[0,0]
    pred = model.predict(x_after[ind+i:ind+i+1], verbose=0)[0,0]
    rec_fit.append(rec_pred)
    fit.append(pred)
rec_fit = np.array(rec_fit)
fit = np.array(fit)

In [ ]:
plt.plot(np.arange(feature_length), x_after[ind], label='after')
plt.plot(np.arange(feature_length)+feature_length, x_after[ind+feature_length], label='after 20')
plt.plot(np.arange(feature_length)+feature_length, rec_fit[feature_length:], label='rec')
plt.plot(np.arange(feature_length)+feature_length, fit, label='one pred')
plt.legend()